In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install tensorflow numpy opencv-python scikit-learn


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
import os
import json
from sklearn.model_selection import train_test_split


In [4]:
videos_path = "/content/drive/My Drive/FF_Face_only_data"  # Modify this if using Google Drive
metadata_path = "/content/drive/My Drive/FF_Face_only_data/metadata.csv"


In [5]:
import pandas as pd

def load_metadata(metadata_path):
    try:
        df = pd.read_csv(metadata_path)
        print("Columns in the metadata file:", df.columns) # Print column names for debugging
        if 'video_path' not in df.columns:
            print("Error: 'video_path' column not found in the metadata file. Please check the file and column names.")
            return None
        if 'label' not in df.columns:
            print("Error: 'label' column not found in the metadata file. Please check the file and column names.")
            return None
        metadata = {row['video_path']: row['label'] for _, row in df.iterrows()}
        return metadata
    except FileNotFoundError:
        print(f"Error: File not found at {metadata_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


metadata = load_metadata(metadata_path)
if metadata:
    print(metadata)  # Verify it loads correctly

Columns in the metadata file: Index(['video_path', 'label'], dtype='object')
{'000.mp4': 'REAL', '000_003.mp4': 'FAKE', '001.mp4': 'REAL', '001_870.mp4': 'FAKE', '002.mp4': 'REAL', '002_006.mp4': 'FAKE', '003.mp4': 'REAL', '003_000.mp4': 'FAKE', '004.mp4': 'REAL', '004_982.mp4': 'FAKE', '005.mp4': 'REAL', '005_010.mp4': 'FAKE', '006.mp4': 'REAL', '006_002.mp4': 'FAKE', '007.mp4': 'REAL', '007_132.mp4': 'FAKE', '008.mp4': 'REAL', '008_990.mp4': 'FAKE', '009.mp4': 'REAL', '009_027.mp4': 'FAKE', '010.mp4': 'REAL', '010_005.mp4': 'FAKE', '011.mp4': 'REAL', '011_805.mp4': 'FAKE', '012.mp4': 'REAL', '012_026.mp4': 'FAKE', '013.mp4': 'REAL', '013_883.mp4': 'FAKE', '014.mp4': 'REAL', '014_790.mp4': 'FAKE', '015.mp4': 'REAL', '015_919.mp4': 'FAKE', '016.mp4': 'REAL', '016_209.mp4': 'FAKE', '017.mp4': 'REAL', '017_803.mp4': 'FAKE', '018.mp4': 'REAL', '018_019.mp4': 'FAKE', '019.mp4': 'REAL', '019_018.mp4': 'FAKE', '020.mp4': 'REAL', '020_344.mp4': 'FAKE', '021.mp4': 'REAL', '021_312.mp4': 'FAKE'

In [6]:
def extract_frames(video_path, label, img_size=(128, 128)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, img_size)
        frame = frame / 255.0  # Normalize
        frames.append(frame)
    cap.release()
    return frames, [label] * len(frames)


In [10]:
from tensorflow.keras.utils import Sequence
import numpy as np
class VideoFrameGenerator(Sequence):
    def __init__(self, videos_path, metadata, batch_size=32, img_size=(128, 128), shuffle=True):
        self.videos_path = videos_path
        self.metadata = list(metadata.items())
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.metadata) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.metadata[index * self.batch_size:(index + 1) * self.batch_size]
        return self.__data_generation(batch_files)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.metadata)

    def extract_frames(self, video_path, img_size):
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or len(frames) >= 10:  # Limit frames per video to save memory
                break
            frame = cv2.resize(frame, img_size)
            frame = frame / 255.0  # Normalize
            frames.append(frame)
        cap.release()
        return np.array(frames)

    def __data_generation(self, batch_files):
        X, y = [], []
        for video_name, label in batch_files:
            video_path = os.path.join(self.videos_path, video_name)
            if os.path.exists(video_path):
                frames = self.extract_frames(video_path, self.img_size)
                for frame in frames:
                    X.append(frame)
                    y.append(1 if label == "FAKE" else 0)
        return np.array(X), np.array(y)


In [11]:
# Create Generators
batch_size = 16
train_generator = VideoFrameGenerator(videos_path, metadata, batch_size=batch_size)
val_generator = VideoFrameGenerator(videos_path, metadata, batch_size=batch_size, shuffle=False)

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
import os
import cv2
def build_model(input_shape=(128, 128, 3)):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the model
model = build_model()
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5230 - loss: 0.7091

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 664s 5s/step - accuracy: 0.5228 - loss: 0.7090 - val_accuracy: 0.5008 - val_loss: 0.6931
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 544s 4s/step - accuracy: 0.4989 - loss: 0.6932 - val_accuracy: 0.4991 - val_loss: 0.6931
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 530s 4s/step - accuracy: 0.5027 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 544s 4s/step - accuracy: 0.4892 - loss: 0.6932 - val_accuracy: 0.5004 - val_loss: 0.6931
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 556s 4s/step - accuracy: 0.5007 - loss: 0.6931 - val_accuracy: 0.5005 - val_loss: 0.6931
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 551s 4s/step - accuracy: 0.4841 - loss: 0.6932 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 538s 4s/step - accuracy: 0.5143 - loss: 0.6931 - val_accuracy: 0.4995 - val_loss: 0.6931
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 540s 4s/step - accuracy: 0.4862 - loss: 0.6932 - val_accuracy: 0.499

In [17]:
loss, accuracy = model.evaluate(val_generator)
print(f"Test Accuracy: {accuracy:.2f}")

124/124 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.4999 - loss: 0.6931
Test Accuracy: 0.50


In [20]:
model.save("/content/deepfake_detector.h5")
print("Model saved successfully!")


Model saved successfully!


Model saved successfully!
